# Análise e manipulação de dados

Os arrays definidos pela biblioteca [NumPy](https://numpy.org/) fornecem funcionalidades essenciais para processamento numérico eficiente em Python. No entanto, estes foram desenhados para lidar com os tipos de conjuntos de dados limpos e bem organizados que são tipicamente usados no contexto de tarefas de computação numérica. No contexto da descoberta e extração de conhecimento de dados, é comum lidar com dados menos estruturados, heterogéneos e que podem ter valores em falta. As limitações dos arrays para a análise e manipulação deste tipo de dados tornam-se rapidamente evidentes. A biblioteca [pandas](https://pandas.pydata.org/) aborda essas limitações, fornecendo uma implementação eficiente de uma tabela de dados (`DataFrame`). As tabelas de dados são basicamente arrays multidimensionais associados a etiquetas para as linhas e colunas e capazes de lidar com tipos heterogéneos e valores em falta. Para além disso, a biblioteca *pandas* implementa várias operações sobre dados que são familiares para os utilizadores de bases de dados e folhas de cálculo. Como as estruturas de dados definidas pela biblioteca *pandas* são construídas em cima de arrays *NumPy*, estas operações são efetuadas de forma eficiente. Isto faz da biblioteca uma ferramenta importante para realizar as tarefas de manipulação de dados que ocupam grande parte do tempo de um cientista de dados.

In [5]:
import numpy as np
import pandas as pd

## Estruturas de dados

A um nível muito básico, as estruturas de dados definidas pela biblioteca *pandas* podem ser vistas como versões melhoradas de arrays *NumPy* nas quais as linhas e colunas são identificadas por etiquetas em vez de um índice baseado na posição. As três estruturas de dados fundamentais definidas pela biblioteca *pandas* são a série (`Series`), a tabela de dados (`DataFrame`) e o índice (`Index`). O índice é uma estrutura interessante por si só, que pode ser vista como um array imutável ou como um conjunto ordenado. No entanto, a sua relevancia deve-se ao seu uso no contexto das outras duas estruturas. Por isso, para simplificar, vamos focar nessas duas e olhar para o índice como algo semelhante a um array.

### Série (`Series`)

Uma série é um array unidimensional de dados indexados.

Séries podem ser criadas de várias formas. Por exemplo, a partir de sequências (ex: listas ou arrays).

In [6]:
data = pd.Series([0.25, 0.5, 0.75, 1])
data

0    0.25
1    0.50
2    0.75
3    1.00
dtype: float64

Uma série combina uma sequência de valores e uma sequência explícita de índices, que podem ser acedidas individualmente através dos atributos `values`e `index`, respetivamente. Os valores são guardados como um array *NumPy*, enquanto os índices são guardados num objeto do tipo `Index` (ou uma das suas subclasses).

In [7]:
data.values

array([0.25, 0.5 , 0.75, 1.  ])

In [8]:
data.index

RangeIndex(start=0, stop=4, step=1)

Tal como nos arrays, é possível aceder aos dados de uma série usando o operador de indexação `[]`:

In [9]:
data[1]

0.5

In [10]:
data[1:3]

1    0.50
2    0.75
dtype: float64

Até agora, uma série parece ser a mesma coisa que um array *NumPy* unidimensional. No entanto, existe uma diferença essencial: enquanto o array tem um índice inteiro definido implicitamente que é usado para aceder aos valores, a série tem um índice definido explicitamente que é associado aos valores. Esta definição explícita do índice confere capacidades adicionais à série. Por exemplo, o índice pode consistir em valores de qualquer tipo e não apenas inteiros.

In [11]:
data = pd.Series([0.25, 0.5, 0.75, 1.0], index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

Independentemente do tipo, o operador de indexação continua a funcionar da mesma forma.

In [12]:
data['b']

0.5

Os valores do índice nem sequer necessitam de ser contíguos ou sequenciais.

In [13]:
data = pd.Series([0.25, 0.5, 0.75, 1.0], index=[2, 5, 3, 7])
data

2    0.25
5    0.50
3    0.75
7    1.00
dtype: float64

In [14]:
data[5]

0.5

Desta perspetiva, uma série pode ser vista como uma especialização de um dicionário. Em Python, um dicionário é uma estrutura que mapeia chaves arbitrárias num conjunto de valores arbitrários. Uma série é uma estrutura que mapeia chaves de um determinado tipo num conjunto de valores que também têm um tipo definido. Esta tipificação é importante pela mesma razão que no caso dos arrays *NumPy*: eficiência das operações realizadas sobre a estrutura.

A analogia da série como um dicionário torna-se ainda mais clara ao construir uma série diretamente a partir de um dicionário:

In [15]:
population_dict = {
    'California': 39538223,
    'Texas': 29145505,
    'Florida': 21538187,
    'New York': 20201249,
    'Pennsylvania': 13002700
}

population = pd.Series(population_dict)
population

California      39538223
Texas           29145505
Florida         21538187
New York        20201249
Pennsylvania    13002700
dtype: int64

Neste caso, o índice é construído a partir do conjunto de chaves do dicionário. O acesso ao valor associado a um índice/chave é feito da mesma forma que num dicionário:  

In [16]:
population['California']

39538223

No entanto, ao contrário de um dicionário, uma série também permite obter todos os valores entre dois índices:

In [17]:
population['California':'Florida']

California    39538223
Texas         29145505
Florida       21538187
dtype: int64

**Nota**: Ao criar uma série a partir de um dicionário, é possível explicitar a ordem e/ou um subconjunto de chaves a usar:

In [18]:
pd.Series({2: 'a', 1: 'b', 3: 'c'}, index=[1, 2])

1    b
2    a
dtype: object

**Nota**: Também é possível criar uma série com um valor constante para todos os índices:

In [19]:
pd.Series(5, index=['John', 'Jane', 'Mary'])

John    5
Jane    5
Mary    5
dtype: int64

### Tabela de dados (`DataFrame`)

Uma tabela de dados é uma sequência de séries que partilham o mesmo índice. Tal como uma série, uma tabela de dados pode ser vista como uma generalização de um array *NumPy* ou como uma especialização de um dicionário. Para exemplificar a criação de uma tabela de dados, vamos começar por criar uma série com as áreas dos estados dos EUA para combinar com a série da população desses estados que criamos anteriormente:

In [20]:
area_dict = {
    'California': 423967,
    'Texas': 695662,
    'Florida': 170312,
    'New York': 141297,
    'Pennsylvania': 119280
}

area = pd.Series(area_dict)
area

California      423967
Texas           695662
Florida         170312
New York        141297
Pennsylvania    119280
dtype: int64

Para criar a tabela de dados, podemos usar um dicionário que associa uma etiqueta a cada uma das séries:

In [21]:
states = pd.DataFrame({'population': population, 'area': area})
states

,population,area
California,39538223,423967
Texas,29145505,695662
Florida,21538187,170312
New York,20201249,141297
Pennsylvania,13002700,119280


Tal como uma série, uma tabela de dados tem um atributo `index` que permite aceder ao índice:

In [22]:
states.index

Index(['California', 'Texas', 'Florida', 'New York', 'Pennsylvania'], dtype='object')

Para além disso, uma tabela de dados tem um atributo `columns` que permite aceder a um índice com as etiquetas das colunas:

In [23]:
states.columns

Index(['population', 'area'], dtype='object')

Logo, uma tabela de dados pode ser vista como uma generalização de um array bidimensional em que quer as linhas, quer as colunas têm um índice explícito que pode ser usado para aceder aos dados. Para além disso, uma tabela de dados também pode ser vista como uma especialização de um dicionário em que as etiquetas das colunas são mapeadas nas séries de dados correspondentes.

In [24]:
states['area']

California      423967
Texas           695662
Florida         170312
New York        141297
Pennsylvania    119280
Name: area, dtype: int64

Para além de um dicionário que associa uma etiqueta a cada uma das séries, podemos criar tabelas de dados de outras formas. Por exemplo, a partir de uma única série:

In [25]:
pd.DataFrame(population, columns=['population'])

,population
California,39538223
Texas,29145505
Florida,21538187
New York,20201249
Pennsylvania,13002700


Também é possível criar uma tabela de dados a partir de um array bidimensional:

In [26]:
pd.DataFrame(np.random.rand(3, 2), columns=['foo', 'bar'], index=['a', 'b', 'c'])

,foo,bar
a,0.409796,0.044031
b,0.388964,0.046356
c,0.446082,0.979269


**Nota**: Neste caso, se um dos ou ambos os índices não forem explicitados, são usados os índices inteiros do array.

In [27]:
pd.DataFrame(np.random.rand(3, 2))

,0,1
0,0.337452,0.036165
1,0.320891,0.133420
2,0.534863,0.494698


Outra opção é criar uma tabela de dados a partir de uma lista de dicionários em que cada um deles representa uma entrada (linha) na tabela:

In [28]:
data = [{'a': i, 'b': 2 * i} for i in range(3)]
pd.DataFrame(data)

,a,b
0,0,0
1,1,2
2,2,4


**Nota**: Ao criar uma tabela de dados, se os índices das séries ou as chaves das entradas não coincidirem, então a tabela vai ter valores em falta, representados por `NaN`.   

In [29]:
s1 = pd.Series(np.random.rand(3), index=['a', 'b', 'c'])
s2 = pd.Series(np.random.rand(3), index=['a', 'd', 'c'])

pd.DataFrame({'s1': s1, 's2': s2})

,s1,s2
a,0.575146,0.339608
b,0.519958,NaN
c,0.894236,0.250093
d,NaN,0.242466


In [30]:
pd.DataFrame([{'a': 1, 'b': 2}, {'b': 3, 'c': 4}])

,a,b,c
0,1.0,2,NaN
1,NaN,3,4.0


## Operações sobre tabelas de dados

A biblioteca *pandas* oferece uma vasta gama de métodos e operações para análise e manipulação de dados a vários níveis. Grande parte do conhecimento sobre essas funcionalidades vai sendo adquirido com a prática e a experiência de lidar os problemas colocados por diferentes conjuntos de dados. Não se espera que alguém domine todas as funcionalidades e é comum consultar a [documentação da biblioteca](https://pandas.pydata.org/docs/index.html) para resolver problemas específicos. No entanto, inicialmente, é importante adquirir uma noção de como usar as operações básicas de transformação de dados e análise estatística, de forma a conseguir dar os primeiros passos na análise e manipulação de conjuntos de dados.

Como exemplo, vamos olhar para um conjunto de dados com informação sobre filmes extraída da [IMDb](https://www.imdb.com/):

In [31]:
import os

data_path = '../data/' if os.path.exists('../data/') else 'https://raw.githubusercontent.com/TheAwesomeGe/DECD/main/data/'

movies_df = pd.read_csv(data_path + 'IMDB-Movie-Data.csv', index_col='Title')

Neste caso, usamos o método `read_csv` para carregar o dataset de um ficheiro CSV e usar os títulos dos filmes como índice.

**Nota**: A biblioteca *pandas* define métodos para carregar datasets em vários formatos, como por exemplo a partir de uma folha de Excel (`read_excel`).  

### Visualizar os dados

A primeira coisa a fazer ao abrir um novo conjunto de dados é olhar para os nomes das colunas e para algumas entradas para ter uma ideia do tipo de informação que é fornecida pelo conjunto de dados.

A representação predefinida do tipo `DataFrame` mostra as 5 primeiras e as 5 últimas entradas do conjunto de dados:

In [32]:
movies_df

,Rank,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
Title,,,,,,,,,,,
Guardians of the Galaxy,1,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
Prometheus,2,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
Split,3,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
Sing,4,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
Suicide Squad,5,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0
...,...,...,...,...,...,...,...,...,...,...,...
Secret in Their Eyes,996,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,NaN,45.0
Hostel: Part II,997,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.54,46.0
Step Up 2: The Streets,998,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.01,50.0


Alternativamente, podemos usar os métodos `head` e `tail` para ver as primeiras ou últimas *n* entradas.

In [33]:
movies_df.head(3)

,Rank,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
Title,,,,,,,,,,,
Guardians of the Galaxy,1,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
Prometheus,2,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
Split,3,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0


**Nota**: Se não for explicitado o número de entradas a mostrar, estes métodos mostram 5 entradas.

In [34]:
movies_df.tail()

,Rank,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
Title,,,,,,,,,,,
Secret in Their Eyes,996,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,NaN,45.0
Hostel: Part II,997,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.54,46.0
Step Up 2: The Streets,998,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.01,50.0
Search Party,999,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,NaN,22.0
Nine Lives,1000,"Comedy,Family,Fantasy",A stuffy businessman finds himself trapped ins...,Barry Sonnenfeld,"Kevin Spacey, Jennifer Garner, Robbie Amell,Ch...",2016,87,5.3,12435,19.64,11.0


### Obter informação sobre os dados

O método `info` fornece os detalhes essenciais sobre um conjunto de dados, como o número de linhas e colunas, o número de valores não nulos, o tipo de dados em cada coluna e a quantidade de memória ocupada:

In [35]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, Guardians of the Galaxy to Nine Lives
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rank                1000 non-null   int64  
 1   Genre               1000 non-null   object 
 2   Description         1000 non-null   object 
 3   Director            1000 non-null   object 
 4   Actors              1000 non-null   object 
 5   Year                1000 non-null   int64  
 6   Runtime (Minutes)   1000 non-null   int64  
 7   Rating              1000 non-null   float64
 8   Votes               1000 non-null   int64  
 9   Revenue (Millions)  872 non-null    float64
 10  Metascore           936 non-null    float64
dtypes: float64(3), int64(4), object(4)
memory usage: 93.8+ KB


Neste caso, facilmente conseguimos perceber que as colunas `Revenue (Millions)` e `Metascore` têm valores em falta.

Conseguir saber os tipos de cada coluna rapidamente é muito útil. Por exemplo, permite perceber se alguns dados foram carregados com o tipo errado (ex: inteiros como cadeias de caracteres).

O atributo `shape` também pode ser usado para descobrir rapidamente o número de entradas e atributos de um conjunto de dados:

In [36]:
movies_df.shape

(1000, 11)

### Limpeza de colunas

Muitas vezes, os conjuntos de dados contêm colunas que não são úteis no contexto do problema que está a ser abordado. Para além disso, é normal as colunas terem nomes verbosos, com símbolos, capitalização variável, espaços, erros, etc.

Para simplificar a análise dos dados e o processo de seleção baseado nos nomes das colunas, podemos fazer alguma limpeza.

Como vimos anteriormente, podemos aceder ao índice com os nomes das colunas de uma tabela de dados usando o atributo `columns`:

In [37]:
movies_df.columns

Index(['Rank', 'Genre', 'Description', 'Director', 'Actors', 'Year',
       'Runtime (Minutes)', 'Rating', 'Votes', 'Revenue (Millions)',
       'Metascore'],
      dtype='object')

Este atributo é útil quando queremos renomear ou descartar colunas, pois permite copiar e colar os nomes das colunas em que estamos interessados. Para além disso, também é útil para perceber porque é que obtivemos um `KeyError` ao tentar selecionar alguns dados com base no nome de uma coluna.

In [38]:
movies_df['Revenue']

KeyError: 'Revenue'

É prática comum renomear as colunas para que os nomes sejam em minúsculas, não contenham caracteres especiais e os espaços sejam substituídos por *underscore*.

Podemos usar o método `rename` para renomear algumas ou todas as colunas usando um dicionário. Vamos usar essa abordagem para remover os parênteses dos nomes das colunas:

In [ ]:
movies_df.rename(columns={
    'Runtime (Minutes)': 'Runtime',
    'Revenue (Millions)': 'Revenue_millions'
}, inplace=True)

movies_df.columns

**Nota**: O argumento `inplace` serve para definir se queremos alterar diretamente a tabela de dados ou obter uma nova tabela com o resultado da aplicação da operação. O comportamento predefinido é não alterar a tabela original.

Também podemos atribuir diretamente uma nova lista de etiquetas ao atributo `columns`:

In [ ]:
movies_df.columns = [
    'rank', 'genre', 'description',
    'director', 'actors', 'year',
    'runtime', 'rating', 'votes',
    'revenue_millions', 'metascore'
]
movies_df.columns

**Nota**: Há uma maneira mais rápida de mudar todos os nomes para minúsculas:

In [ ]:
movies_df.columns = [c.lower() for c in movies_df.columns]
movies_df.columns

Compreensões de lista (e dicionário) são muito úteis em conjunto com as operações da biblioteca *pandas* e ao trabalhar com dados no geral.

Para selecionar o conjunto de colunas que nos interessa, podemos usar o operador de indexação `[]` com uma lista dos nomes das colunas que queremos:

In [ ]:
selected_df = movies_df[['genre', 'director', 'year', 'runtime', 'rating', 'votes', 'revenue_millions', 'metascore']]
selected_df.columns

In [ ]:
selected_df.head(3)

Alternativamente, podemos descartar as colunas que não nos interessam usando o método `drop`:

In [ ]:
movies_df.drop(columns=['rank', 'description', 'actors'], inplace=True)

In [ ]:
movies_df.head(3)

### Valores em falta

Ao explorar um conjunto de dados, é possível encontrar valores em falta ou nulos. As representações mais comuns para estes valores desconhecidos ou não existententes são `None` ou `numpy.NaN`.

As abordagens mais comuns para lidar com valores em falta são:

1. Apagar as linhas ou colunas com valores em falta
2. Prencher os valores em falta com valores obtidos usando uma técnica chamada imputação

Para calcular o número de valores em falta num conjunto de dados, podemos começar por verificar quais as células da tabela de dados que têm uma das representações para valores desconhecidos. Para isso, podemos usar o método `isnull`:

In [ ]:
movies_df.isnull()

Este método devolve uma tabela de dados em que cada célula tem um valor booleano que indica se o valor está em falta. Por si só isto não é muito útil. No entanto, podemos usar uma função de agregação, neste caso a soma (`sum`), para obter um resultado mais interessante:

In [ ]:
movies_df.isnull().sum()

Assim podemos concluir que existem **128** valores em falta na coluna `revenue_millions` e **64** valores em falta na coluna `metascore`.

#### Remoção de valores em falta

Cientistas e analistas de dados são regularmente confrontados com o dilema de descartar ou imputar valores em falta. Esta é uma decisão que requer conhecimento dos dados e do contexto em que eles são utilizados. No entanto, normalmente, só é recomendada a remoção das entradas com valores em falta quando estas representam uma porção insignificante do conjunto de dados total. Pelo contrário, só é recomendada a remoção de atributos com valores em falta quando estes estão em maioria.

O método `dropna` pode ser usado para descartar valores em falta de uma tabela de dados. O comportamento predefinido é apagar todas as **entradas** que têm pelo menos um valor em falta:

In [ ]:
movies_df.dropna()

No caso deste conjunto de dados, seriam removidas `1000 - 838 = 162` entradas ao aplicar esta operação. Isto é um desperdício, uma vez que existem dados que podem ser importantes nas outras colunas dessas entradas.

Para descartar atributos com valores em falta podemos usar o método `dropna` com o argumento `axis='columns'` ou `axis=1`:

In [ ]:
movies_df.dropna(axis='columns')

Neste caso, são mantidas as 1000 entradas, mas as colunas `revenue_millions` e `metascore` são removidas.


#### Imputação

Imputação (o processo de substituir valores em falta por valores representativos) é uma técnica convencional de engenharia de atributos usada para evitar descartar entradas com valores em falta. Na prática, consiste em substituir os valores em falta por valores representativos, como por exemplo a média ou a mediana do atributo no conjunto de dados.

Como exemplo, vamos usar esta estratégia para lidar com os valores em falta na coluna `revenue_millions`:

In [ ]:
revenue = movies_df['revenue_millions']
revenue_mean = revenue.mean()
revenue_mean

Agora que já temos a média, podemos usar o método `fillna` para preencher os valores em falta com esse valor:

In [ ]:
revenue.fillna(revenue_mean, inplace=True)

**Nota**: O método foi chamado sobre a série `revenue` pois só queremos preencher os valores em falta para esse atributo e não todos os valores em falta na tabela de dados.

In [ ]:
movies_df.isnull().sum()

**Nota**: Imputar uma todos os valores em falta de uma coluna com o mesmo valor é um exemplo básico da aplicação técnica. Uma ideia melhor seria usar uma imputação mais granular. Por exemplo, podiamos calcular a média para cada género de filme ou para cada realizador e usar esses valores para preencher os valores em falta em entradas com as mesmas características.

### Análise de atributos

O método `describe` pode ser usado para obter um resumo da distribuição dos atributos contínuos de uma tabela de dados:

In [ ]:
movies_df.describe()

O método `describe` também pode ser aplicado sobre uma série. Se essa série representar um atributo categórico, a informação obtida consiste no número de entradas, número de valores diferentes, qual o valor mais comum e a frequência desse valor:

In [ ]:
movies_df['genre'].describe()

Isto diz-nos que a coluna `genre` tem 207 valores diferentes, sendo o mais comum `Action/Adventure/Sci-Fi`, que aparece 50 vezes.

O método `value_counts` pode ser usado para obter a frequência dos valores numa coluna:

In [ ]:
movies_df['genre'].value_counts().head(10)

O método `corr` pode ser usado para analisar a correlação entre cada par de atributos contínuos no conjunto de dados:

In [ ]:
movies_df.corr(numeric_only=True)

Números positivos indicam uma correlação positiva (quando um aumenta, o outro também aumenta) e números negativos indicam uma correlação inversa (quando um aumenta, o outro decresce). O valor 1.0 indica uma correlação perfeita.

Olhando para a tabela de correlações, podemos, por exemplo, ver que cada variável tem uma correlação perfeita consigo própria. No entanto, esta informação é óbvia e, por isso, pouco interessante. Por outro lado, uma correlação de 0.6 entre as variáveis `votes` e `revenue_millions` é uma observação mais interessante.

Analisar tabelas de correlação entre atributos é útil, por exemplo, para identificar quais os atributos mais relacionados com um outro atributo de interesse. Entre outras coisas, esta informação pode depois ser usada para fazer uma seleção de atributos para reduzir a dimensionalidade.

### Seleção de dados

Já vimos anteriormente que podemos selecionar dados com base no nome das colunas/atributos. Se usarmos o operador de indexação `[]` com o nome duma coluna, obtemos a série corresponde. Se usarmos uma lista, obtemos uma tabela de dados com as colunas incluídas na lista.

In [ ]:
genre_col = movies_df['genre']
type(genre_col)

In [ ]:
genre_col = movies_df[['genre']]
type(genre_col)

Também é possível selecionar linhas/entradas específicas. Para isso existem dois métodos:

- `loc` - seleciona por nome (valor do índice)
- `iloc`- seleciona por posição numérica no índice

O nosso conjunto de dados está indexado pelo título dos filmes. Por isso, podemos usar o método `loc` para obter a entrada correspondente ao filme com um determinado título:

In [39]:
movies_df.loc['Prometheus']

Rank                                                                  2
Genre                                          Adventure,Mystery,Sci-Fi
Description           Following clues to the origin of mankind, a te...
Director                                                   Ridley Scott
Actors                Noomi Rapace, Logan Marshall-Green, Michael Fa...
Year                                                               2012
Runtime (Minutes)                                                   124
Rating                                                              7.0
Votes                                                            485820
Revenue (Millions)                                               126.46
Metascore                                                          65.0
Name: Prometheus, dtype: object

Podemos obter a mesma entrada usando o método `iloc` com a posição do filme no índice:

In [ ]:
movies_df.iloc[1]

Estes métodos também podem ser usados para obter uma sequência de entradas contíguas da mesma forma que numa lista ou array *NumPy*:

In [ ]:
movies_df.iloc[1:4]

In [ ]:
movies_df.loc['Prometheus':'Sing']

**Nota**: Uma distinção importante entre os dois métodos quando são usados para obter múltiplas entradas é que num deles o intervalo é aberto à direita e no outro é fechado. No caso do método `iloc`, o intervalo é aberto à direita, tal como na indexação de listas e arrays. Por isso, o filme na posição 4 não foi selecionado. No caso do método `loc`, o intervalo é fechado. Por isso, o filme Sing foi selecionado.

É importante saber selecionar atributos ou entradas específicas, mas mais interessante que isso é conseguir selecionar dados que satisfazem uma determinada condição. Por exemplo, filmes realizados pelo Ridley Scott ou filmes com uma classificação igual ou superior a 8.0.

Para fazer seleções deste tipo, podemos aplicar condições booleanas sobre as colunas de uma tabela de dados:

In [ ]:
rs_movies = movies_df['director'] == 'Ridley Scott'
rs_movies.head()

O resultado da aplicação duma condição deste tipo é semelhante ao obtido quando aplicamos o método `isnull`. Neste caso temos uma série de valores booleanos que indicam se o Ridley Scott é o realizador de cada um dos filmes.

Podemos usar métodos de agregação sobre este resultado para descobrir, por exemplo, quantos dos filmes foram realizados pelo Ridley Scott:

In [ ]:
rs_movies.sum()

Mas o nosso objectivo era obter as entradas correspondentes aos filmes realizados pelo Ridley Scott...

Para isso, temos de usar a condição (ou o resultado dela) com o operador de indexação sobre a tabela de dados:

In [ ]:
movies_df[rs_movies].head()

In [ ]:
movies_df[movies_df['director'] == 'Ridley Scott'].head()

Para quem está habituado a trabalhar com bases de dados SQL, pode ajudar interpretar este tipo de seleção como:

> select * from movies_df where director = Ridley Scott

As condições também podem ser aplicadas sobre atributos numéricos:

In [ ]:
movies_df[movies_df['rating'] >= 8.6].head()

É possível fazer seleções mais complexas recorrendo aos operadores lógicos `|` (ou) e `&` (e).

Por exemplo, podemos selecionar filmes realizados pelo Ridley Scott OU pelo Christopher Nolan:


In [ ]:
movies_df[(movies_df['director'] == 'Ridley Scott') | (movies_df['director'] == 'Christopher Nolan')].head()

**Nota**: A utilização de parênteses à volta de cada condição é necessária para o interpretador de Python saber avaliar corretamente.

A mesma seleção pode ser feita de forma mais simples usando o método `isin`:

In [ ]:
movies_df[movies_df['director'].isin(['Christopher Nolan', 'Ridley Scott'])].head()

As seleções condicionais podem envolver condições sobre várias colunas e usar estatísticas dos dados, o que as torna muito poderosas. Por exemplo, é possível selecionar filmes que estrearam entre 2005 e 2010 e com uma classificação acima de 8.0, mas que tiveram uma receita abaixo da média:

In [ ]:
movies_df[
    ((movies_df['year'] >= 2005) & (movies_df['year'] <= 2010))
    & (movies_df['rating'] > 8.0)
    & (movies_df['revenue_millions'] < movies_df['revenue_millions'].mean())
]

## Transformação de atributos

Existem múltiplas razões que levam à necessidade de transformar os atributos de conjunto de dados de alguma forma. Por exemplo:

- A representação de um determinado atributo não é a mais adequada no contexto do problema que queremos abordar
- As abordagens de extração de conhecimento que queremos aplicar não são compatíveis com um determinado tipo de atributo
- Queremos agrupar atributos ou gerar novos atributos com base nos existentes

Como exemplo, vamos transformar o atributo `rating` num atributo categórico que tem o valor `'good'` se a classificação for igual ou superior a 8.0 e o valor `'bad'` caso contrário. Para isso, vamos começar por criar a função que faz essa transformação:

In [ ]:
def rating_function(x):
    if x >= 8.0:
        return 'good'
    else:
        return 'bad'

É possível iterar sobre uma série ou tabela de dados da mesma forma que sobre uma lista e usar essa abordagem para aplicar a função a todas as entradas:

In [ ]:
pd.Series([rating_function(r) for r in movies_df['rating']], index=movies_df.index)

No entanto, essa é uma operação que se torna lenta em conjuntos de dados de grande dimensão. Uma alternativa mais eficiente é usar o método `apply`:

In [ ]:
movies_df['rating'].apply(rating_function)

**Nota**: O método `apply` é mais eficiente pois usa vetorização, isto é, a função é aplicada a todas as entradas de uma só vez.

Para adicionar o novo atributo ou substituir o existente, podemos usar a analogia da tabela de dados como um dicionário:

In [ ]:
movies_df['rating_category'] = movies_df['rating'].apply(rating_function)
movies_df.head(2)

**Nota**: Muitas vezes é útil usar uma função anónima como argumento do método `apply`. Estas têm a sintaxe `lambda <argumentos>: <expressão>`. Por exemplo, a transformação anterior também poderia ser feita da seguinte forma:

In [ ]:
movies_df['rating'].apply(lambda x: 'good' if x >= 8.0 else 'bad')

Se olharmos para a informação dada pelo método `info` podemos ver que atributo que criámos tem o tipo `object`, tal como os outros atributos que são cadeias de caracteres:

In [ ]:
movies_df.info()

Podemos explicitar que se trata de um atributo categórico usando o método `astype`:

In [ ]:
movies_df['rating_category'] = movies_df['rating_category'].astype('category')
movies_df.info()

A função `factorize` pode ser usada para transformar os valores dum atributo categórico em valores inteiros:

In [ ]:
pd.factorize(movies_df['rating_category'])

Para exemplificar a criação de um novo atributo a partir de uma combinação dos existentes, vamos gerar um novo atributo que diz a variação entre as duas classificações de um filme (`rating` e `metascore`):

In [ ]:
# We divide the metascore by 10 so that both ratings are in the same scale
movies_df['rating_difference'] = movies_df.apply(lambda x: x['rating'] - x['metascore'] / 10, axis='columns')
movies_df.head()

Neste caso, estamos a aplicar o método `apply` sobre a tabela de dados e a explicitar o argumento `axis='columns'` de forma a ter acesso a todos os atributos e podermos usá-los na geração do novo atributo.

**Nota**: Como as séries são construídas em cima de arrays *NumPy*, este atributo também pode ser obtido fazendo as operações diretamente sobre os dois atributos:

In [ ]:
movies_df['rating'] - movies_df['metascore'] / 10

**Nota**: Uma área em que o método `apply` é usado exaustivamente é o processamento de língua natural. Nesse contexto é necessário aplicar uma panóplia de funções de limpeza e manipulação de texto para preparar os dados para aplicação de abordagens de aprendizagem automática.

## Considerações finais

A capacidade de analisar, explorar, transformar e visualizar dados é essencial na ciência de dados. Os vários passos deste processo ocupam uma grande parte do tempo de quem trabalha nesta área. Como tal, é importante que seja possível reproduzir de forma fácil o processo de análise e manipulação feito sobre um determinado conjunto de dados e/ou que, pelo menos, o seu resultado seja guardado.

Tal como para a leitura de conjuntos de dados em vários formatos, a biblioteca *pandas* fornece um conjunto de métodos para guardar conjuntos de dados nesses mesmos formatos. Por exemplo, podemos usar o método `to_csv` para guardar o nosso conjunto de dados processado num ficheiro CSV:

In [ ]:
movies_df.to_csv('IMDB-Movie-Data-Processed.csv')

Tal como referido anteriormente, a biblioteca *pandas* oferece uma vasta gama de métodos e operações para análise e manipulação de dados a vários níveis. Neste tutorial cobrimos apenas uma pequena parte da funcionalidade disponibilizada pela biblioteca: a funcionalidade básica que é usada em quase todas as tarefas de análise e manipulação de dados. Para explorar alguns temas mais a fundo, recomendamos os [tutoriais da biblioteca pandas](https://pandas.pydata.org/pandas-docs/stable/tutorials.html) e o [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/).  